In [1461]:
#!L
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim 
import torch.nn.functional as F
import torchvision.datasets as datasets 
import torchvision.transforms as transforms   
import torchvision.models as models

import numpy as np

In [1462]:
#!L
import code

import importlib
importlib.reload(code)

<module 'code' from '/home/jupyter/work/resources/code.py'>

In [1463]:
#!L
seed = 228
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

# eps = 0.004
# temper = 100.0
dataset = 'imagenet-c'
experiment_name = 'r50_ic_odin'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# net = models.densenet161(pretrained=True).to(device)
# net = models.resnet18(pretrained=True).to(device)

net = models.resnet50(pretrained=False).to(device)
net.load_state_dict(torch.load('re0.pth'))

net.eval()
filename = f'_{experiment_name}.txt'

params = {'test_ratio_in' : 0.99}

if dataset == 'imagenet-o':
    params['test_ratio_out'] = 0.75
if dataset == 'imagenet-r':
    params['test_ratio_out'] = 0.98333333
if dataset == 'imagenet-g':
    params['test_ratio_out'] = 1.0
if dataset == 'imagenet-a':
    params['test_ratio_out'] = 0.93333333
if dataset == 'imagenet-c':
    params['test_ratio_out'] = 0.99

information = f'resnet-50 + imagenet-c + odin'

in_loader_test, out_loader_test, in_loader_gridsearch, out_loader_gridsearch = code.get_data_gridsearch(params, in_path='data', out_path=f'./{dataset}', filename=filename)

print('Done!')

cuda


Inlier dataset total length: 50000
Outlier dataset total length: 50000


Inlier dataset test length: 49500
Outlier dataset test length: 49500
Inlier dataset gridsearch length: 500
Outlier dataset gridsearch length: 500
Validation:
inlier test first 10 indices: [3476, 21289, 43731, 42751, 44948, 44513, 29319, 4412, 25411, 26328]
outlier test first 10 indices: [40501, 9136, 39963, 25701, 46206, 47583, 19062, 9589, 47065, 17599]
inlier gridsearch first 10 indices: [13685, 37855, 16426, 30378, 11578, 36179, 18448, 7856, 21234, 34914]
outlier gridsearch first 10 indices: [15363, 19841, 16691, 40684, 14486, 25461, 43392, 42742, 7576, 12729]


Done!


In [1464]:
#!L
# inlier test first 10 indices: [3476, 21289, 43731, 42751, 44948, 44513, 29319, 4412, 25411, 26328]
# outlier test first 10 indices: [1627, 58, 909, 146, 1697, 1590, 1954, 1844, 1604, 0]
# inlier gridsearch first 10 indices: [13685, 37855, 16426, 30378, 11578, 36179, 18448, 7856, 21234, 34914]
# outlier gridsearch first 10 indices: [1408, 1487, 231, 1525, 983, 654, 361, 1825, 924, 619]

In [1465]:
#!L
%%time

eps, temper = code.gridsearch_odin(in_loader_gridsearch, out_loader_gridsearch, net, device, filename, information)

CPU times: user 2h 17min 52s, sys: 6min 32s, total: 2h 24min 25s
Wall time: 2h 17min 36s


In [1466]:
#!L
print(eps, temper)

0.0 500.0


In [1467]:
#!L
in_norms = code.odin(eps, temper, in_loader_test, net, device)
code.get_stats(in_norms, 'inliers:', filename=filename)

out_norms = code.odin(eps, temper, out_loader_test, net, device)
code.get_stats(out_norms, 'outliers:', filename=filename)

current iter: 0


current iter: 5000


current iter: 10000


current iter: 15000


current iter: 20000


...

current iter: 30000


current iter: 35000


current iter: 40000


current iter: 45000


min = -0.0010586637072265148
max = -0.001005804631859064
median = -0.0010149203008040786
mean = -0.0010158978536997855
std = 5.03745330301273e-06


In [1468]:
#!L
# code.compare_histograms(in_norms, out_norms, in_name='imagenet', out_name=f'{dataset}', filename=filename)

In [1469]:
#!L
# code.calc_all_metrics(in_norms, out_norms, information=information, filename=filename)

In [1470]:
#!L
code.save_and_bootstrap(experiment_name, in_norms, out_norms, bootstrap_num=100)

bootstrap iter 0! 49500, 49500
0.9692175082134475, 0.1681897233201581, 0.9708767073722294, 0.9686842123485939


bootstrap iter 10! 49500, 49500
0.9691787697173757, 0.16660655333826063, 0.970747078331937, 0.9686966132674057


bootstrap iter 20! 49500, 49500
0.9688184291398837, 0.16951777515413882, 0.9704972780912021, 0.9682401082304694


bootstrap iter 30! 49500, 49500
0.9700248154269973, 0.16482491582491585, 0.9715816784679081, 0.9693306878437895


bootstrap iter 40! 49500, 49500
0.9702759336802368, 0.1615218855218855, 0.9719014083281265, 0.9697535225673257


...

bootstrap iter 60! 49500, 49500
0.9693523436384043, 0.16548677789127228, 0.9707241566657638, 0.9685802918206485


bootstrap iter 70! 49500, 49500
0.9689706040199979, 0.169659243032737, 0.970728553401085, 0.968252089814543


bootstrap iter 80! 49500, 49500
0.9695245915722885, 0.1680550079104296, 0.9713500089936414, 0.9690866053433876


bootstrap iter 90! 49500, 49500
0.9690525852464035, 0.16912534435261706, 0.9708775334505668, 0.9685439350119827


roc-auc: 0.9694258582409959 ± 0.00047769947209320507
fpr: 0.16685146145174024 ± 0.0029706250577529463
pr_auc_pos: 0.9710702767230059 ± 0.0004667225432543668
pr_auc_neg: 0.9688349658705079 ± 0.0005389214178663557
